# Sarsa learning on Mountain car with linear function approximation

In this exercise you will be coding semi-gradient sarsa learning algorithm on the mountain car task. In the mountain car task, a car is stuck in the valley between two mountains and the goal is to steer it to the top of the right mountain. The car is underpowered to drive up the right mountain directly. Instead the car has to first move away from the goal and then, by applying full throttle it will build up enough inertia to reach the top of the right mountain. The agent's state space consists of position and velocity (continuous space). The agent can pick one of the three available actions (throttle forward, throttle backward, zero throttle). The reward for reaching the top of the right mountain is 0 and a reward of -1 is given to the agent everywhere else. You can find additional task description [here](https://gym.openai.com/envs/MountainCar-v0/). Complete the code to train RL agent solve this task.

In [1]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


## Tile coding

We use a tile coding to convert the continuous state space into a feature vector. For now, you can treat the below code as a black box and just think of it as a function that will provide you the features when you pass it a state and an action. In tile coding, the entire state space is divided into multiple grids. Given a state, the tile coder returns the indicies of the tiles corresponding to agent's location. We use multiple tiles with different offsets to obtain the agent's location. We get a binary feature vector corresponding to the agent's state.

In [1]:
"""
Tile Coding Software version 3.0beta
by Rich Sutton
based on a program created by Steph Schaeffer and others
External documentation and recommendations on the use of this code is available in the 
reinforcement learning textbook by Sutton and Barto, and on the web.
These need to be understood before this code is.

This software is for Python 3 or more.

This is an implementation of grid-style tile codings, based originally on
the UNH CMAC code (see http://www.ece.unh.edu/robots/cmac.htm), but by now highly changed. 
Here we provide a function, "tiles", that maps floating and integer
variables to a list of tiles, and a second function "tiles-wrap" that does the same while
wrapping some floats to provided widths (the lower wrap value is always 0).

The float variables will be gridded at unit intervals, so generalization
will be by approximately 1 in each direction, and any scaling will have 
to be done externally before calling tiles.

Num-tilings should be a power of 2, e.g., 16. To make the offsetting work properly, it should
also be greater than or equal to four times the number of floats.

The first argument is either an index hash table of a given size (created by (make-iht size)), 
an integer "size" (range of the indices from 0), or nil (for testing, indicating that the tile 
coordinates are to be returned without being converted to indices).
"""

basehash = hash

class IHT:
    "Structure to handle collisions"
    def __init__(self, sizeval):
        self.size = sizeval                        
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        "Prepares a string for printing whenever this object is printed"
        return "Collision table:" + \
               " size:" + str(self.size) + \
               " overfullCount:" + str(self.overfullCount) + \
               " dictionary:" + str(len(self.dictionary)) + " items"

    def count (self):
        return len(self.dictionary)
    
    def fullp (self):
        return len(self.dictionary) >= self.size
    
    def getindex (self, obj, readonly=False):
        d = self.dictionary
        if obj in d: return d[obj]
        elif readonly: return None
        size = self.size
        count = self.count()
        if count >= size:
            if self.overfullCount==0: print('IHT full, starting to allow collisions')
            self.overfullCount += 1
            return basehash(obj) % self.size
        else:
            d[obj] = count
            return count

def hashcoords(coordinates, m, readonly=False):
    if type(m)==IHT: return m.getindex(tuple(coordinates), readonly)
    if type(m)==int: return basehash(tuple(coordinates)) % m
    if m==None: return coordinates

from math import floor, log
from itertools import zip_longest

def tiles (ihtORsize, numtilings, floats, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    qfloats = [floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append( (q + b) // numtilings )
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles

def tileswrap (ihtORsize, numtilings, floats, wrapwidths, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints, wrapping some floats"""
    qfloats = [floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q, width in zip_longest(qfloats, wrapwidths):
            c = (q + b%numtilings) // numtilings
            coords.append(c%width if width else c)
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles

num_tiles = 2048 # number of tiles
num_tilings = 8

iht = IHT(num_tiles)

def get_features(state, action):
  """
  Returns the features 
  """
  tile_id = tiles(iht, num_tilings, [num_tilings*state[0]/(0.6+1.2), num_tilings*state[1]/(0.07+0.07)], [action]) # return the indices of the tiles corresponding to agent's location
  features = np.zeros(feature_length)
  features[tile_id] = 1 # set features corresponding to the above indicies to 1
  return features

In [2]:
import gym
import numpy as np
from pprint import pprint as pp
from matplotlib import cm
import sys

OpenAI gym is a toolkit that has many commonly used reinforcement learning environments. [check](https://gym.openai.com/docs/) this out for a detailed instructions on how to set it up and use it.

We will make use of the following commands:


```
env = gym.make(env_name) # sets up the environment
c_s = env.reset() # resets the environment and returns the starting state for the agent
n_s, rew, done, _ = env.step(action) # applies action in the current state and returns the next state, reward. done is true if the episode has ended.
```



In [4]:
Tenv = gym.make('MountainCar-v0') # make gym environment

We will now initialize our FA weights and set the values of hyperparameters.


In [17]:
feature_length = 2048
weights = np.zeros(feature_length) # this is our FA weights, initialized to 0
epsilon = 0.1 
# amount of exploration in your policy
num_episodes = 100 # total number of episodes to train the agent
lr_rate = 0.1/num_tilings #learning rate to update the weights (divided by num_tilings to account for tilings)
discount = 0.99 # discount factor (gamma)

In [18]:
'''
This is our policy. An epsilon greedy policy that return a random action during exploration and samples an action corresponding to max action value during exploitation.
'''
def get_action(state):
    if np.random.binomial(1, epsilon) == 1:
        # return random action - exploration
        return np.random.choice([0,1,2]) # [0,1,2] are the available actions to the agent
    values = []
    for action in [0,1,2]: #loop for each step of episode
        values.append(weights.T.dot(get_features(state, action)))
    # return action corresponding to the max action value - exploitation
    return np.random.choice([action_ for action_, value_ in enumerate(values) if value_ == np.max(values)])

Refer to Section 10.1 from [Sutton and Barto book](http://www.incompleteideas.net/book/RLbook2020.pdf) to fill up the following code.

## Answer the following questions:


1.   Plot the training curves i.e., plot the returns obtained per episode in the y-axis and the episode number in the x-axis.
2.   What happens if you set epsilon to 0 while training? Also experiment with different values of epsilon (0.1, 0.5). What do you observe?
3.   What happens if you change the learning rate?


In [ ]:
'''
Fill up the following code.
'''
velocities = []
positions = []

for epi in range(num_episodes):
  # initialize env
  env = gym.make('MountainCar-v0')
  # get current action
  c_s = env.reset()
  pos, vel = c_s
  positions.append(pos)
  velocities.append(vel)
 
  c_a = get_action(c_s)
  #c_av = 1
  #n_av = 1
  done = False
  while not done:
    # get next_state, reward, done, _ from env
    env.render()
    n_s, rew, done, _ = env.step(c_a) 
        # applies action in the current state and returns the next state, reward. done is true if the episode has ended.
    # get next_action
    n_a = get_action(n_s)
    c_features = get_features(c_s, c_a)
    # get features corresponding to the current state and the next state
    n_features = get_features(n_s, n_a)
    # compute action values of the current and next states
    c_av = np.dot(weights, c_features)
    n_av = np.dot(weights, n_features)
    # compute td error
    td_err = rew + (discount*n_av) - c_av
    # update weights
    weights = weights + lr_rate*td_err*c_features
    
    #print(n_features)
    
    # update current state and current action
    c_s = n_s
    c_a = n_a
    
    pos, vel = c_s
    positions.append(pos)
    velocities.append(vel)
    
#print(weights)
#env.render()

Exception ignored in: <bound method Viewer.__del__ of <gym.envs.classic_control.rendering.Viewer object at 0x000001DB68057978>>
Traceback (most recent call last):
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "C:\Users\rapha\Anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: (<weakref at 0x000001DB6D08D6D8; to 'Win32Window' at 0x000001DB680D8BE0>,)
Exception ignored in: <bound method Viewer.__del__ of <gym.envs.classic_control.rendering.Viewer object at 0x000001D

Exception ignored in: <bound method Viewer.__del__ of <gym.envs.classic_control.rendering.Viewer object at 0x000001DB685114E0>>
Traceback (most recent call last):
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "C:\Users\rapha\Anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: (<weakref at 0x000001DB7019E958; to 'Win32Window' at 0x000001DB68511C88>,)
Exception ignored in: <bound method Viewer.__del__ of <gym.envs.classic_control.rendering.Viewer object at 0x000001D

Exception ignored in: <bound method Viewer.__del__ of <gym.envs.classic_control.rendering.Viewer object at 0x000001DB70173E80>>
Traceback (most recent call last):
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "C:\Users\rapha\Anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: (<weakref at 0x000001DB6D08D4A8; to 'Win32Window' at 0x000001DB70173F60>,)
Exception ignored in: <bound method Viewer.__del__ of <gym.envs.classic_control.rendering.Viewer object at 0x000001D

    self.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "C:\Users\rapha\Anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: (<weakref at 0x000001DB6D070778; to 'Win32Window' at 0x000001DB70061F28>,)
Exception ignored in: <bound method Viewer.__del__ of <gym.envs.classic_control.rendering.Viewer object at 0x000001DB70061BE0>>
Traceback (most recent call last):
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "C:\Users\rapha\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 83,

In [16]:
pp(range(num_episodes))

range(0, 100)


TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'

In [13]:
'''
Write the plot code here
'''
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

positions, velocities = np.meshgrid(positions, velocities)
pp(num_episodes)
z = range(num_episodes)
z = np.reshape(z, (2,2))
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.contour3D(positions, velocities, z, cmap=cm.coolwarm)


MemoryError: Unable to allocate 1.13 EiB for an array with shape (404010000, 404010000) and data type float64